In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import timeit
import pickle
import sys
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, \
                            precision_recall_curve, roc_curve, accuracy_score
from sklearn.exceptions import NotFittedError

In [ ]:
X_test = np.load('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/XDAWN/X_test.npy')
X_train = np.load('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/XDAWN/X_train.npy')
y_train=np.load('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/Y_train.npy')
y_test = np.reshape(pd.read_csv('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/true_labels.csv', header=None).values, 3400)
X_test.shape,X_train.shape,Y_train.shape,Y_test.shape

((3400, 210), (5440, 210), (5440,), (3400,))

In [ ]:
pip install lightgbm

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
lgbm = LGBMClassifier()

In [ ]:
lgbm.fit(X_train,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
y_preds=lgbm.predict(X_test)
cr = classification_report(y_test,y_preds)
print(cr)

              precision    recall  f1-score   support

           0       0.65      0.31      0.42       989
           1       0.77      0.93      0.84      2411

    accuracy                           0.75      3400
   macro avg       0.71      0.62      0.63      3400
weighted avg       0.73      0.75      0.72      3400



In [ ]:
confusion_matrix(y_test,y_preds)

array([[ 308,  681],
       [ 167, 2244]])

After Hyperparameter tuning using Optuna

In [ ]:
!pip install optuna


     |████████████████████████████████| 307kB 7.4MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 174kB 13.1MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 112kB 14.0MB/s 
     |████████████████████████████████| 143kB 14.6MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11136 sha256=b2f175df1082448104b37d8ed6c61bcba49380639e3bdf1173c52e18678ff693
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


In [ ]:
pip install catboost

     |████████████████████████████████| 69.2MB 55kB/s 


In [ ]:
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
 
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))
class_weights

{0: 1.7106918238993711, 1: 0.7064935064935065}

In [ ]:
def objective(trial):
    #X_train, X_test, y_train, y_test = train_test_split(train,test, test_size=0.3)

    param = {
        "l2_leaf_reg" : trial.suggest_int("l2_leaf_reg", 1,15),
        #"objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 10),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "class_weights" :class_weights,
        #"bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        
        "od_wait":  trial.suggest_int("od_wait", 5, 10),
        "od_type": "Iter",    #overfitting detection
        "use_best_model": True,
        "eval_metric": "F1",
       
    }

    

    gbm = cb.CatBoostClassifier(**param)

    gbm.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0, early_stopping_rounds=100)

    preds = gbm.predict(X_test)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_test, pred_labels)
    return accuracy

In [ ]:
study = optuna.create_study(direction="maximize") #create_study creates a bigger space where traiing occurs and importanve is give to maximize accuracy
study.optimize(objective, timeout=600)

[I 2021-06-08 08:42:48,596] A new study created in memory with name: no-name-8c65a59f-03bd-4ef6-b0d2-469ab5092806
[I 2021-06-08 08:43:46,279] Trial 0 finished with value: 0.7076470588235294 and parameters: {'l2_leaf_reg': 14, 'colsample_bylevel': 0.07303229778133417, 'depth': 9, 'boosting_type': 'Ordered', 'od_wait': 9}. Best is trial 0 with value: 0.7076470588235294.
[I 2021-06-08 08:43:53,890] Trial 1 finished with value: 0.7308823529411764 and parameters: {'l2_leaf_reg': 8, 'colsample_bylevel': 0.049288870538189394, 'depth': 6, 'boosting_type': 'Plain', 'od_wait': 6}. Best is trial 1 with value: 0.7308823529411764.
[I 2021-06-08 08:43:56,042] Trial 2 finished with value: 0.6385294117647059 and parameters: {'l2_leaf_reg': 10, 'colsample_bylevel': 0.013849027023335585, 'depth': 2, 'boosting_type': 'Ordered', 'od_wait': 6}. Best is trial 1 with value: 0.7308823529411764.
[I 2021-06-08 08:44:02,863] Trial 3 finished with value: 0.7282352941176471 and parameters: {'l2_leaf_reg': 14, 'col

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial
print

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 39
Best trial:
  Value: 0.7520588235294118
  Params: 
    l2_leaf_reg: 4
    colsample_bylevel: 0.09019602817875072
    depth: 10
    boosting_type: Plain
    od_wait: 5


In [ ]:
y_preds=lgbm.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
cls = classification_report(y_test,y_preds)
print(cls)
print("------------COnfusion Matrix---------------")
confusion_matrix(y_test,y_preds)

              precision    recall  f1-score   support

           0       0.65      0.31      0.42       989
           1       0.77      0.93      0.84      2411

    accuracy                           0.75      3400
   macro avg       0.71      0.62      0.63      3400
weighted avg       0.73      0.75      0.72      3400

------------COnfusion Matrix---------------


array([[ 308,  681],
       [ 167, 2244]])